# Modelo implantado na sprint 3 

Abaixo estamos executando o mesmo modelo que a Sprint 3. A ideia principal é predizer novas imagens com o dataset novo [dataset_inteli_test]().


## Obter dados no drive

In [ ]:
from google.colab import drive
import pandas as pd
import random
import os
import time
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
from keras import layers, models, Input, regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
drive.mount('/content/drive')

## Carregamento de dados - Sem Data Augmentation


In [ ]:
images = []
masks = []

for path in glob('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*'):
  images.append(path + '/image.tif')
  masks.append(path + '/mask.png')

In [ ]:
# Função para carregar e pré-processar uma imagem e sua máscara
def load_and_preprocess_image(image_path, mask_path, target_size):

    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalização entre 0 e 1

    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0  # Normalização entre 0 e 1

    return image, mask

In [ ]:
# Lista para armazenar imagens e máscaras pré-processadas
images_processed = []
masks_processed = []

count = 1
# Carregar e pré-processar todas as imagens e máscaras
for img_path, mask_path in zip(images, masks):
    print(count)
    count += 1
    img, mask = load_and_preprocess_image(img_path, mask_path, target_size=(256, 256))
    images_processed.append(img)
    masks_processed.append(mask)

# Converter para arrays numpy
images_processed = np.array(images_processed)
masks_processed = np.array(masks_processed)


In [ ]:
images_processed

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images_processed, masks_processed, test_size=0.3, random_state=42)

## Definição de parâmetros e modelo

In [ ]:
class CyclicLR(Callback):
    def __init__(self, base_lr=1e-4, max_lr=1e-3, step_size=2000., mode='triangular'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.iterations = 0
        self.history = {}

    def clr(self):
        cycle = np.floor(1 + self.iterations / (2 * self.step_size))
        x = np.abs(self.iterations / self.step_size - 2 * cycle + 1)
        lr = self.base_lr + (self.max_lr - self.base_lr) * max(0, (1 - x))
        if self.mode == 'triangular2':
            lr = lr / float(2 ** (cycle - 1))
        elif self.mode == 'exp_range':
            lr = lr * (0.999 ** self.iterations)
        return lr

    def on_train_begin(self, logs=None):
        logs = logs or {}
        tf.keras.backend.set_value(self.model.optimizer.lr, self.base_lr)

    def on_batch_end(self, batch, logs=None):
        self.iterations += 1
        lr = self.clr()
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        self.history.setdefault('lr', []).append(lr)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

# Função de callbacks
def get_callbacks():
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    clr = CyclicLR(base_lr=1e-4, max_lr=1e-3, step_size=2000., mode='triangular2')
    return [early_stopping, reduce_lr, clr]

In [ ]:
# Função para calcular a sigmoide e converter para 0 ou 1 o output
class ThresholdLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.where(inputs < 0.5, 0.0, 1.0)

# Função para calcular o Dice Coefficient
def dice_coefficient(y_train, y_val):
    smooth = 1e-6
    intersection = tf.reduce_sum(y_train * y_val)
    dice_coefficient = (2. * intersection + smooth) / (tf.reduce_sum(y_train) + tf.reduce_sum(y_val) + smooth)
    return dice_coefficient

# Função de perda de Dice
def dice_loss(y_train, y_val):
    return 1 - dice_coefficient(y_train, y_val)

# Função para calcular a penalidade adicional
def penalty_loss(y_train, y_val, penalty_weight):
    # Calcular a penalidade considerando a diferença entre y_train e y_val
    penalty = tf.reduce_sum(tf.abs(y_train - y_val))
    # Multiplicar a penalidade pelo peso da penalidade
    weighted_penalty = penalty_weight * penalty
    return weighted_penalty

# Função de perda combinada
def combined_loss(y_train, y_val, penalty_weight):
    # Perda padrão (por exemplo, perda de entropia cruzada binária)
    standard_loss = tf.keras.losses.binary_crossentropy(y_train, y_val)
    # Dice Loss
    dice = dice_loss(y_train, y_val)
    # Penalidade adicional
    penalty = penalty_loss(y_train, y_val, penalty_weight)
    # Perda total = perda padrão + penalidade + Dice Loss
    total_loss = standard_loss + penalty + dice
    return total_loss

# Métrica de acurácia customizada
def custom_accuracy(y_train, y_val):
    # Calcular a acurácia considerando uma tolerância de 0.5 na predição
    y_val_binary = tf.round(y_val)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_train, y_val_binary), tf.float32))
    return accuracy

In [ ]:
def Unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    reg = regularizers.L2(0.001)

    # Encoder (contraction path)
    conv1 = layers.Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=reg)(inputs)
    conv1 = layers.Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=reg)(conv1)
    drop1 = layers.Dropout(0.3)(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(drop1)

    conv2 = layers.Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=reg)(pool1)
    conv2 = layers.Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=reg)(conv2)
    drop2 = layers.Dropout(0.3)(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(drop2)

    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=reg)(pool2)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=reg)(conv3)
    drop3 = layers.Dropout(0.3)(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(drop3)

    # Bottleneck
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=reg)(pool3)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=reg)(conv4)
    drop4 = layers.Dropout(0.3)(conv4)

    # Decoder (expansion path)
    up5 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(drop4)
    merge5 = layers.concatenate([conv3, up5], axis=3)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=reg)(merge5)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=reg)(conv5)

    up6 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv5)
    merge6 = layers.concatenate([conv2, up6], axis=3)
    conv6 = layers.Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=reg)(merge6)
    conv6 = layers.Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=reg)(conv6)

    up7 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = layers.concatenate([conv1, up7], axis=3)
    conv7 = layers.Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=reg)(merge7)
    conv7 = layers.Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=reg)(conv7)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv7)  # Saída com um canal (máscara binária)

    threshold_output = ThresholdLayer()(outputs)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Criar modelo U-Net
model = Unet(input_shape=(256, 256, 3))
model.summary()

## Treino do modelo - GPU

In [ ]:
callbacks = get_callbacks()
with tf.device('/gpu:0'):
    model = Unet(input_shape=(256, 256, 3))
    # Compilar o modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss=lambda y_train, y_val: combined_loss(y_train, y_val, 0.001), metrics=[custom_accuracy])

    # Calcula o tempo de treino
    start_time = time.time()

    # Definições
    max_epochs = 200
    batch_size = 16

    # Treinar o modelo
    H = model.fit(X_train, y_train, validation_data=(X_val, y_val), steps_per_epoch=30, epochs=max_epochs, batch_size=batch_size, callbacks=callbacks)

    # Para o cronômetro e salva o tempo de treino
    training_time_gpu = time.time() - start_time

# Avaliação do modelo - GPU

In [ ]:
with tf.device('/gpu:0'):
    # Mostrando resultados
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.epoch, H.history["loss"], label="train_loss")
    plt.plot(H.epoch, H.history["val_loss"], label="val_loss")
    plt.plot(H.epoch, H.history["custom_accuracy"], label="train_acc")
    plt.plot(H.epoch, H.history["val_custom_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy - GPU")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Calcula o tempo de inferência
    start_time = time.time()

    # Avaliar o modelo nos dados de teste
    results = model.evaluate(X_val, y_val)

    # Para o cronômetro e salva o tempo de treino
    inference_time_gpu = time.time() - start_time

    print("Test Loss - GPU:", results[0])
    print("Test Accuracy - GPU:", results[1])

    # Prever máscaras usando o modelo
    predicted_masks = model.predict(X_val)

In [ ]:
with tf.device('/gpu:0'):
    # Obter métricas de precisão e perda do histórico de treinamento
    acc = H.history['custom_accuracy']
    val_acc = H.history['val_custom_accuracy']
    loss = H.history['loss']
    val_loss = H.history['val_loss']

    # Número de épocas
    epochs = range(1, len(acc) + 1)

    # Plotar precisão do conjunto de treino e validação
    plt.plot(epochs, acc, 'r', label='Precisão do Conjunto de Treino')
    plt.plot(epochs, val_acc, 'b', label='Precisão do Conjunto de Validação')
    plt.title('Precisão do Conjunto de Treino e Validação - GPU')
    plt.xlabel('Épocas')
    plt.ylabel('Precisão')
    plt.legend()
    plt.show()

    # Plotar perda do conjunto de treino e validação
    plt.plot(epochs, loss, 'r', label='Perda do Conjunto de Treino')
    plt.plot(epochs, val_loss, 'b', label='Perda do Conjunto de Validação')
    plt.title('Perda do Conjunto de Treino e Validação - GPU')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.show()

In [ ]:
with tf.device('/gpu:0'):
    print(epochs)
    print('Tempo de treino GPU (segundos): ', training_time_gpu)
    print('Tempo de treino GPU por época (segundos): ', training_time_gpu / max_epochs)
    print('Tempo de inferência GPU (segundos): ', inference_time_gpu)

    # Plotando o tempo de treinamento e inferência
    plt.subplot(1, 2, 2)
    plt.bar(['GPU Training'], [training_time_gpu], color=['orange'])
    plt.bar(['GPU Inference'], [inference_time_gpu], color=['orange'])
    plt.ylabel('Time (seconds)')
    plt.title('Training and Inference Time - GPU')

    plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Gerar as saídas do modelo para um conjunto de entradas de teste
    saidas_modelo = model.predict(X_val)

    # Iterar sobre cada saída do modelo
    for i in range(len(X_val)):
        # Obter a entrada correspondente e a saída real
        img_entrada = X_val[i]
        img_saida_real = y_val[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('Entrada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - GPU')
        plt.axis('off')

        plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Métricas do parceiro de Projeto:

    # Lista para armazenar os scores de IoU
    iou_scores = []
    # Calcular IoUs e determinar predições corretas
    correct_predictions = 0
    iou_threshold = 0.5
    for mask, result in zip(y_val, img_saida_modelo):
        intersection = np.logical_and(mask, result)
        union = np.logical_or(mask, result)
        iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0
        iou_scores.append(iou_score)
        # Verificar se a predição é considerada correta (IoU >= threshold)
        if iou_score >= iou_threshold:
            correct_predictions += 1
        print('IoU é: ' + str(iou_score))
    # Calcular a média dos IoUs
    iou_mean = np.mean(iou_scores)
    print('Média dos IoU - GPU:', iou_mean)
    # Calcular Coverage Ratio (CovR)
    total_predictions = len(iou_scores)
    covr = correct_predictions / total_predictions if total_predictions > 0 else 0
    print('Coverage Ratio (CovR) - GPU:', covr)

# Avaliação do Modelo com Imagens Desconhecidas

Esta seção apresenta os resultados da avaliação do modelo utilizando um conjunto de imagens inéditas, ou seja, imagens que não foram utilizadas durante o treinamento. O objetivo principal é analisar o desempenho e comportamento do modelo frente a dados desconhecidos. 

A métrica principal utilizada nesta etapa inicial é a Intersection over Union (IoU), a qual será complementada com métricas adicionais na próxima sprint, proporcionando uma análise mais abrangente da performance do modelo. 


In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def load_masks(masks_dir, target_size=(600, 600), crop_size=(200, 200), final_size=(256, 256)):
    """
    Carrega as máscaras (masks) de segmentação a partir de um diretório.

    Args:
    masks_dir (str): O diretório contendo as máscaras.
    target_size (tuple): Tamanho alvo para redimensionamento inicial.
    crop_size (tuple): Tamanho do recorte para segmentação.
    final_size (tuple): Tamanho final após redimensionamento.

    Returns:
    list: Lista de máscaras normalizadas.
    list: Lista de nomes de arquivo de máscara correspondentes.
    """
    masks = []
    mask_filenames = []
    ordered_masks = sorted(os.listdir(masks_dir))
    for mask_name in ordered_masks:
        mask_path = os.path.join(masks_dir, mask_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is not None:
            if mask.shape[:2] != target_size:
                mask = cv2.resize(mask, target_size[::-1], interpolation=cv2.INTER_NEAREST)
            cropped_masks = crop_image(mask, crop_size)
            resized_masks = [cv2.resize(crop, final_size[::-1], interpolation=cv2.INTER_NEAREST) for crop in cropped_masks]
            normalized_masks = [crop / 255.0 for crop in resized_masks]
            masks.extend(normalized_masks)
            mask_filenames.extend([f"{os.path.splitext(mask_name)[0]}_crop_{i}" for i in range(len(normalized_masks))])
        else:
            print(f"Falha ao carregar a máscara: {mask_path}")
    return masks, mask_filenames

def load_images(image_dir, target_size=(600, 600), crop_size=(200, 200), final_size=(256, 256)):
    """
    Carrega as imagens a partir de um diretório.

    Args:
    image_dir (str): O diretório contendo as imagens.
    target_size (tuple): Tamanho alvo para redimensionamento inicial.
    crop_size (tuple): Tamanho do recorte para segmentação.
    final_size (tuple): Tamanho final após redimensionamento.

    Returns:
    list: Lista de imagens normalizadas.
    list: Lista de nomes de arquivo de imagem correspondentes.
    """
    images = []
    image_filenames = []
    ordered_images = sorted(os.listdir(image_dir))
    for image_name in ordered_images:
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if image.shape[:2] != target_size:
                image = cv2.resize(image, target_size[::-1])
            cropped_images = crop_image(image, crop_size)
            resized_images = [cv2.resize(crop, final_size[::-1]) for crop in cropped_images]
            normalized_images = [crop / 255.0 for crop in resized_images]
            images.extend(normalized_images)
            image_filenames.extend([f"{os.path.splitext(image_name)[0]}_crop_{i}" for i in range(len(normalized_images))])
        else:
            print(f"Falha ao carregar a imagem: {image_path}")
    return images, image_filenames

def crop_image(image, crop_size=(200, 200)):
    """
    Realiza o recorte da imagem em múltiplas partes.

    Args:
    image (np.array): A imagem a ser recortada.
    crop_size (tuple): Tamanho do recorte.

    Returns:
    list: Lista de partes recortadas da imagem.
    """
    crops = []
    for i in range(0, image.shape[0], crop_size[0]):
        for j in range(0, image.shape[1], crop_size[1]):
            crop = image[i:i+crop_size[0], j:j+crop_size[1]]
            if crop.shape[0] == crop_size[0] and crop.shape[1] == crop_size[1]:
                crops.append(crop)
    return crops

# Diretórios
masks_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/masks'
image_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli_test/tci_tifs'

# Carregar imagens e máscaras
masks_test, mask_filenames = load_masks(masks_dir)
images_test, image_filenames = load_images(image_dir)

In [ ]:
# Certificando que o conjunto possui o tamanho correto, 3 imagens divididas em 9 partes cada.
len(images_test)

In [ ]:
# Converter a lista de imagens em um array numpy para ser usado como entrada para o modelo
images_test = np.array(images_test)

# Gerar as saídas do modelo para um conjunto de entradas de teste
saidas_modelo = model.predict(images_test)

In [ ]:
with tf.device('/gpu:0'):
    # Iterar sobre cada saída do modelo
    for i in range(len(images_test)):
        # Obter a entrada correspondente e a saída real
        img_entrada = images_test[i]
        img_saida_real = masks_test[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('Entrada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - GPU')
        plt.axis('off')

        plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Métricas do parceiro de Projeto:

    # Lista para armazenar os scores de IoU
    iou_scores = []
    # Calcular IoUs e determinar predições corretas
    correct_predictions = 0
    iou_threshold = 0.5
    for mask, result in zip(masks_test, saidas_modelo):
        intersection = np.logical_and(mask, result)
        union = np.logical_or(mask, result)
        iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0
        iou_scores.append(iou_score)
        # Verificar se a predição é considerada correta (IoU >= threshold)
        if iou_score >= iou_threshold:
            correct_predictions += 1
        print('IoU é: ' + str(iou_score))
    # Calcular a média dos IoUs
    iou_mean = np.mean(iou_scores)
    print('Média dos IoU - GPU:', iou_mean)
    # Calcular Coverage Ratio (CovR)
    total_predictions = len(iou_scores)
    covr = correct_predictions / total_predictions if total_predictions > 0 else 0
    print('Coverage Ratio (CovR) - GPU:', covr)

Podemos observar que o conjunto de imagens novo teve um ganho relativo até se comparado com as de validação e treino. Cerca de um crescimento de 7 pontos percentuais.

# Pós-processamento de Imagens: Operações Morfológicas e CRFs

No pós-processamento de imagens, buscamos aprimorar os resultados de segmentação inicial, tornando-os mais precisos e visualmente coerentes. Duas técnicas comuns utilizadas para esse fim são as **operações morfológicas** e os **Campos Aleatórios Condicionais (CRFs)**. Este documento explora essas técnicas, seus fundamentos teóricos, benefícios e aplicações.

## Operações Morfológicas

Operações morfológicas são técnicas de processamento de imagem que se baseiam na forma ou estrutura de objetos dentro de uma imagem. Utilizando um elemento estruturante, essas operações podem modificar a geometria de regiões binárias da imagem para eliminar ruídos ou destacar características específicas.

### Tipos de Operações

1. **Erosão**: Remove pixels nas bordas dos objetos.
2. **Dilatação**: Adiciona pixels às bordas dos objetos.
3. **Abertura**: Erosão seguida de dilatação, útil para remover pequenos objetos.
4. **Fechamento**: Dilatação seguida de erosão, útil para fechar pequenos buracos.

### Benefícios

- **Redução de Ruído**: Abertura e fechamento são eficazes na eliminação de pequenos ruídos sem alterar significativamente a forma dos objetos.
- **Suavização de Bordas**: Dilatação e erosão podem suavizar bordas irregulares, melhorando a qualidade visual da segmentação.
  
Para mais detalhes sobre operações morfológicas, consulte o artigo que utilizamos como base: [Understanding Morphological Image Processing and its Operations](https://towardsdatascience.com/understanding-morphological-image-processing-and-its-operations-7bcf1ed11756).

## Campos Aleatórios Condicionais (CRFs)

Os CRFs são modelos gráficos não direcionados que consideram as dependências entre pixels vizinhos para melhorar a segmentação de imagens. Diferente de outros classificadores, os CRFs são discriminativos e consideram as relações espaciais entre os pixels.

### Tipos de CRFs

1. **Linear CRF**: Adequado para sequências lineares, como texto.
2. **Grid CRF**: Conecta cada pixel aos seus vizinhos imediatos em uma grade.
3. **Dense CRF**: Conecta cada pixel a todos os outros pixels, ideal para capturar relações de longo alcance.

### Benefícios

- **Aprimoramento de Bordas**: CRFs são eficazes na recuperação de bordas suaves e contornos detalhados em imagens segmentadas.
- **Redução de Ambiguidade**: Considerando a vizinhança, os CRFs ajudam a resolver ambiguidades na classificação de pixels.

O uso de CRFs densos é particularmente notável pela sua eficiência e precisão, conforme descrito no artigo "Efficient Inference in Fully Connected CRFs with Gaussian Edge Potentials" por Krähenbühl e Koltun. Para um estudo detalhado, consulte o artigo: [Dense Conditional Random Field](https://medium.com/lis-computer-vision-blogs/dense-conditional-random-field-dfdeb6655005).

### Referências

- DHAWAN, Aashish; BODANI, Pankaj; GARG, Vishal. Post Processing of Image Segmentation using Conditional Random Fields. 2019. Disponível em: [ResearchGate](https://www.researchgate.net/publication/355020007_Post_Processing_of_Image_Segmentation_using_Conditional_Random_Fields).
- Towards Data Science. Understanding Morphological Image Processing and its Operations. Disponível em: [Medium](https://towardsdatascience.com/understanding-morphological-image-processing-and-its-operations-7bcf1ed11756).
- LIS Computer Vision Blogs. Dense Conditional Random Field. Disponível em: [Medium](https://medium.com/lis-computer-vision-blogs/dense-conditional-random-field-dfdeb6655005).

In [ ]:
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_bilateral

## GridSerach


Abaixo, implementamos uma técnica de GridSearch para encontrar os melhores parâmetros para o algoritmo de CRF e para as operações morfológicas. Utilizamos a métrica de **Jaccard Score**, também conhecida como **Intersection Over Union (IoU)**, para avaliar os hiperparâmetros que proporcionam os melhores resultados.

### Métrica de Avaliação: Jaccard Score (IoU)
A métrica **Jaccard Score** é amplamente utilizada na avaliação de segmentação de imagens, pois mede a similaridade entre os conjuntos previstos e os conjuntos reais. É calculada como a razão entre a interseção e a união dos conjuntos, proporcionando uma medida clara de precisão.


In [ ]:
pip install tqdm

### Gridsearch para encontrar os melhores parâmetros de CRF

In [ ]:
from tqdm import tqdm

def postprocess_mask_with_crf(image, mask, crf_params):
    softmax = np.stack([1 - mask, mask], axis=-1)
    softmax = np.ascontiguousarray(softmax.transpose(2, 0, 1))
    image = np.ascontiguousarray(image)
    image_uint8 = (image * 255).astype(np.uint8)
    d = dcrf.DenseCRF2D(image.shape[1], image.shape[0], 2)
    unary = unary_from_softmax(softmax)
    d.setUnaryEnergy(unary)
    d.addPairwiseGaussian(sxy=crf_params['sxy'], compat=crf_params['compat'])
    d.addPairwiseBilateral(sxy=crf_params['sxy_bilateral'], srgb=crf_params['srgb'], rgbim=image_uint8, compat=crf_params['compat_bilateral'])
    Q = d.inference(5)
    refined_mask = np.argmax(Q, axis=0).reshape((image.shape[0], image.shape[1]))
    return refined_mask / 255.0

def calculate_iou(true_mask, pred_mask):
    true_mask_bin = (true_mask > 0).astype(np.uint8)
    pred_mask_bin = (pred_mask > 0).astype(np.uint8)
    return jaccard_score(true_mask_bin.flatten(), pred_mask_bin.flatten(), average='binary')

# Espaço de busca de hiperparâmetros do CRF
crf_param_grid = {
    'sxy': [1, 3, 5],
    'compat': [3, 5, 10],
    'sxy_bilateral': [49, 81],
    'srgb': [3, 10, 13],
    'compat_bilateral': [3, 10, 20]
}

best_iou_crf = 0
best_crf_params = None

# Calcular o número total de iterações para CRF
total_iterations_crf = (len(crf_param_grid['sxy']) * len(crf_param_grid['compat']) *
                        len(crf_param_grid['sxy_bilateral']) * len(crf_param_grid['srgb']) *
                        len(crf_param_grid['compat_bilateral']))

# Realizar busca em grade para encontrar os melhores hiperparâmetros do CRF
with tqdm(total=total_iterations_crf, desc="Grid Search CRF") as pbar:
    for sxy in crf_param_grid['sxy']:
        for compat in crf_param_grid['compat']:
            for sxy_bilateral in crf_param_grid['sxy_bilateral']:
                for srgb in crf_param_grid['srgb']:
                    for compat_bilateral in crf_param_grid['compat_bilateral']:
                        crf_params = {
                            'sxy': sxy,
                            'compat': compat,
                            'sxy_bilateral': sxy_bilateral,
                            'srgb': srgb,
                            'compat_bilateral': compat_bilateral
                        }
                        # Aplicar pós-processamento e calcular IoU
                        iou_scores = []
                        for i in range(len(saidas_modelo)):
                            refined_mask = postprocess_mask_with_crf(images_test[i], saidas_modelo[i].squeeze(), crf_params)
                            iou = calculate_iou(masks_test[i], refined_mask)
                            iou_scores.append(iou)
                        avg_iou = np.mean(iou_scores)
                        if avg_iou > best_iou_crf:
                            best_iou_crf = avg_iou
                            best_crf_params = crf_params
                        pbar.update(1)

print("Melhores parâmetros CRF:", best_crf_params)
print("Melhor IoU CRF:", best_iou_crf)


#### Aplicando apenas CRF e comparando a qualidade da imagem


In [ ]:
best_crf_params = {
    'sxy': 5,
    'compat': 3,
    'sxy_bilateral': 81,
    'srgb': 10,
    'compat_bilateral': 20
}

# Pós-processar as máscaras preditas com CRF denso
saidas_modelo_postprocessed_with_best_crf = np.array([postprocess_mask_with_crf(images_test[i], saidas_modelo[i].squeeze(), best_crf_params) for i in range(len(saidas_modelo))])

In [ ]:
# Converter as máscaras de teste para binário (0 e 1)
masks_test_binary = (np.array(masks_test) > 0).astype(np.uint8)
saidas_modelo_postprocessed_with_best_crf_binary = (saidas_modelo_postprocessed_with_best_crf > 0).astype(np.uint8)

# Calcular Jaccard Score
jaccard_scores = [jaccard_score(masks_test_binary[i].flatten(), saidas_modelo_postprocessed_with_best_crf_binary[i].flatten(), average='binary') for i in range(len(masks_test))]
print(f"Jaccard Score: {np.mean(jaccard_scores)}")


In [ ]:
with tf.device('/gpu:0'):
    # Iterar sobre cada saída do modelo
    for i in range(len(images_test)):
        # Obter a entrada correspondente e a saída real
        img_entrada = masks_test[i]
        img_saida_real = saidas_modelo[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo_postprocessed_with_best_crf[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('saida real')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída modelo')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - pós')
        plt.axis('off')

        plt.show()

### Gridsearch para encontrar os melhores operações morfológicas

In [ ]:
# Cálculo para operações morfológicas
def postprocess_mask_with_morphology(mask, morph_params):
    refined_mask = (mask * 255).astype(np.uint8)
    kernel = cv2.getStructuringElement(morph_params['shape'], morph_params['kernel_size'])
    if morph_params['operation'] == 'open':
        refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_OPEN, kernel, iterations=morph_params['iterations'])
    elif morph_params['operation'] == 'close':
        refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_CLOSE, kernel, iterations=morph_params['iterations'])
    elif morph_params['operation'] == 'dilate':
        refined_mask = cv2.dilate(refined_mask, kernel, iterations=morph_params['iterations'])
    elif morph_params['operation'] == 'erode':
        refined_mask = cv2.erode(refined_mask, kernel, iterations=morph_params['iterations'])
    return refined_mask / 255.0

def calculate_iou(true_mask, pred_mask):
    true_mask_bin = (true_mask > 0).astype(np.uint8)
    pred_mask_bin = (pred_mask > 0).astype(np.uint8)
    return jaccard_score(true_mask_bin.flatten(), pred_mask_bin.flatten(), average='binary')

# Espaço de busca de hiperparâmetros das operações morfológicas
morph_param_grid = {
    'kernel_size': [(3, 3), (5, 5), (7, 7)],
    'iterations': [1, 2, 3],
    'operation': ['open', 'close', 'dilate', 'erode'],
    'shape': [cv2.MORPH_RECT, cv2.MORPH_ELLIPSE, cv2.MORPH_CROSS]
}

best_iou_morph = 0
best_morph_params = None

# Calcular o número total de iterações para operações morfológicas
total_iterations_morph = (len(morph_param_grid['kernel_size']) * len(morph_param_grid['iterations']) *
                          len(morph_param_grid['operation']) * len(morph_param_grid['shape']))

# Realizar busca em grade para encontrar os melhores hiperparâmetros das operações morfológicas
with tqdm(total=total_iterations_morph, desc="Grid Search Morph") as pbar:
    for kernel_size in morph_param_grid['kernel_size']:
        for iterations in morph_param_grid['iterations']:
            for operation in morph_param_grid['operation']:
                for shape in morph_param_grid['shape']:
                    morph_params = {
                        'kernel_size': kernel_size,
                        'iterations': iterations,
                        'operation': operation,
                        'shape': shape
                    }
                    # Aplicar pós-processamento e calcular IoU
                    iou_scores = []
                    for i in range(len(saidas_modelo)):
                        refined_mask_crf = postprocess_mask_with_crf(images_test[i], saidas_modelo[i].squeeze(), best_crf_params)
                        refined_mask = postprocess_mask_with_morphology(refined_mask_crf, morph_params)
                        iou = calculate_iou(masks_test[i], refined_mask)
                        iou_scores.append(iou)
                    avg_iou = np.mean(iou_scores)
                    if avg_iou > best_iou_morph:
                        best_iou_morph = avg_iou
                        best_morph_params = morph_params
                    pbar.update(1)

print("Melhores parâmetros de Morfologia:", best_morph_params)
print("Melhor IoU Morph:", best_iou_morph)

# Teste: Operação Morfológica Antes ou Depois do CRF
iou_scores_before_crf = []
iou_scores_after_crf = []

for i in range(len(saidas_modelo)):
    # Aplicar operação morfológica antes do CRF
    morph_mask_before_crf = postprocess_mask_with_morphology(saidas_modelo[i].squeeze(), best_morph_params)
    refined_mask_before_crf = postprocess_mask_with_crf(images_test[i], morph_mask_before_crf, best_crf_params)
    iou_before_crf = calculate_iou(masks_test[i], refined_mask_before_crf)
    iou_scores_before_crf.append(iou_before_crf)

    # Aplicar operação morfológica depois do CRF
    refined_mask_crf = postprocess_mask_with_crf(images_test[i], saidas_modelo[i].squeeze(), best_crf_params)
    morph_mask_after_crf = postprocess_mask_with_morphology(refined_mask_crf, best_morph_params)
    iou_after_crf = calculate_iou(masks_test[i], morph_mask_after_crf)
    iou_scores_after_crf.append(iou_after_crf)

avg_iou_before_crf = np.mean(iou_scores_before_crf)
avg_iou_after_crf = np.mean(iou_scores_after_crf)

print("IoU Médio - Operação Morfológica Antes do CRF:", avg_iou_before_crf)
print("IoU Médio - Operação Morfológica Depois do CRF:", avg_iou_after_crf)


#### Avaliando operações morfológicas

In [ ]:
best_morph_params = {
    'kernel_size': (3,3),
    'iterations': 1,
    'operation': 'dilate',
    'shape': cv2.MORPH_CROSS
}

# Pós-processar as máscaras preditas com CRF denso
saidas_modelo_postprocessed_with_morphology = np.array([postprocess_mask_with_morphology(saidas_modelo[i].squeeze(), best_morph_params) for i in range(len(saidas_modelo))])

In [ ]:
# Converter as máscaras de teste para binário (0 e 1)
saidas_modelo_postprocessed_with_morphology_binary = (saidas_modelo_postprocessed_with_morphology > 0).astype(np.uint8)

# Calcular Jaccard Score
jaccard_scores = [jaccard_score(masks_test_binary[i].flatten(), saidas_modelo_postprocessed_with_morphology_binary[i].flatten(), average='binary') for i in range(len(masks_test))]
print(f"Jaccard Score: {np.mean(jaccard_scores)}")


## Pós processamento completo

In [ ]:
def morphology_then_crf(image, mask, morph_params, crf_params):
    morph_mask = postprocess_mask_with_morphology(mask, morph_params)
    refined_mask = postprocess_mask_with_crf(image, morph_mask, crf_params)
    return refined_mask

In [ ]:
# Pós-processar as máscaras preditas com CRF denso
saidas_modelo_postprocessed = np.array([morphology_then_crf(images_test[i], saidas_modelo[i].squeeze(), best_morph_params, best_crf_params) for i in range(len(saidas_modelo))])

In [ ]:
# Converter as máscaras de teste para binário (0 e 1)
saidas_modelo_postprocessed_binary = (saidas_modelo_postprocessed > 0).astype(np.uint8)

# Calcular Jaccard Score
jaccard_scores = [jaccard_score(masks_test_binary[i].flatten(), saidas_modelo_postprocessed_binary[i].flatten(), average='binary') for i in range(len(masks_test))]
print(f"Jaccard Score: {np.mean(jaccard_scores)}")


Ao avaliarmos a saída das imagens do modelo, percebemos que, para as próximas sprints, há necessidade de um refinamento maior do algoritmo de CRF e uma melhora circunstancial da máscara predita pelo modelo, permitindo alcançarmos um maior nível de generalização para o modelo.

In [ ]:
with tf.device('/gpu:0'):
    # Iterar sobre cada saída do modelo
    for i in range(len(images_test)):
        # Obter a entrada correspondente e a saída real
        img_entrada = masks_test[i]
        img_saida_real = saidas_modelo[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo_postprocessed[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('saida real')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída modelo')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - pós')
        plt.axis('off')

        plt.show()

## TESTE COM MODELOS DE TREINO

In [ ]:
# Gerar as saídas do modelo para um conjunto de entradas de teste
saidas_modelo_treino = model.predict(X_val)


In [ ]:
# Pós-processar as máscaras preditas com CRF denso
imagens_treino_postprocessed = np.array([morphology_then_crf(X_val[i], saidas_modelo_treino[i].squeeze(), best_morph_params, best_crf_params) for i in range(len(saidas_modelo_treino))])

In [ ]:
# Converter as máscaras de teste para binário (0 e 1)
masks_treino = (np.array(y_val) > 0).astype(np.uint8)
imagens_treino_postprocessed_binary = (np.array(X_val) > 0).astype(np.uint8)

# Calcular Jaccard Score
jaccard_scores = [jaccard_score(masks_treino[i].flatten(), imagens_treino_postprocessed_binary[i].flatten(), average='binary') for i in range(len(masks_treino))]
print(f"Jaccard Score: {np.mean(jaccard_scores)}")


In [ ]:
X_val.shape, y_val.shape, imagens_treino_postprocessed.shape

In [ ]:
with tf.device('/gpu:0'):
    # Iterar sobre cada saída do modelo
    for i in range(len(X_val)):
        # Obter a entrada correspondente e a saída real
        img_entrada = X_val[i]
        img_saida_real = y_val[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo_treino[i]
        img_saida_modelo_post = imagens_treino_postprocessed[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída Modelo')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo_post.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - PÓS')
        plt.axis('off')

        plt.show()